In [ ]:
# !pip install qiskit 
# !pip install qiskit-aer
# !pip install pylatexenc

# Importer les bibliothèques nécessaires

In [ ]:
from qiskit_aer import Aer

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister #, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import XGate, ZGate

import os, sys
sys.path.append(os.getcwd())

from utils import get_disjunction_control_state, get_disjunction_qubits

# Les sept clauses du problème de Pincus

Une clause de la forme

$
    (x_0 \lor x_2 \lor \neg x_3)
$

est écrite : 
```
clause = {'x0' : True, 'x2' : True, 'x3' : False}
````

Rappel : 

* x0 = Peureux
* x1 = Heureux
* x2 = Malade
* x3 = Bruyant


In [ ]:
clauses = []

# Si un habitant est en santé et courageux, il est aussi bruyant. 
# (x2 ⌵ x0 ⌵ x3)  
clauses.append({'x2' : True, 'x0' : True, 'x3' : True})

# Si un habitant est peureux et discret, il est aussi heureux.
# (¬x0 ⌵ x3 ⌵ x1)  
# écrire la définition de la clause ici

# Si un habitant est en santé et bruyant, il est aussi heureux.
# (x2 ⌵ ¬x3 ⌵ x1)  
# écrire la définition de la clause ici

# Si un habitant est en heureux et discret, il est aussi en santé.
# (¬x1 ⌵ x3 ⌵ ¬x2)  
# écrire la définition de la clause ici

# Si un habitant est en peureux et en santé, il est aussi triste.
# (¬x0 ⌵ x2 ⌵ ¬x1)  
# écrire la définition de la clause ici

# Si un habitant est en triste et malade, il est aussi peureux.
# (x1 ⌵ ¬x2 ⌵ x0)  
# écrire la définition de la clause ici

# Si un habitant est en peureux et bruyant, il est aussi en santé.
# (¬x0 ⌵ ¬x3 ⌵ ¬x2)  
# écrire la définition de la clause ici

# Données connues du problème

In [ ]:
# x0 : peureux, x1 : heureux, x2 : malade, x3 : bruyant
nb_variables = 4
# 7 clauses
nb_clauses = len(clauses)

# Création d'une fonction pour transformer la disjonction en un circuit

Nous allons définir une fonction qui prend en entrée une disjonction et qui la transforme en circuit quantique. 

Par exemple, la formule : 

$
    (x_0 \lor x_1 \lor \neg x_2)
$

serait transformée en la série de portes suivantes : 

<img src="disj_circuit.png" alt= “” width="15%" height="15%">

Chaque disjonction est transformée en un circuit contenant autant de qubits qu'il y a de variables dans la formule, en
plus d'un qubit ancillaire associé à la clause en question. Le circuit est ensuite converti en une porte qui peut être
ajoutée à un circuit (notamment à l'oracle), en spécifiant les bons qubits. 

In [ ]:
# Transformer une disjonction en une porte
def logical_disjunction_to_gate(disj_clause):
    # Nombre de variables dans la clause
    nb_disj_variables = len(disj_clause)
    
    # Nombre de qubits dans la porte de disjonction (nb_variables + 1 qubit ancillaire)
    nb_qubits = nb_disj_variables + 1
    disj_qc = QuantumCircuit(nb_qubits)
    qubits = disj_qc.qubits
    
    # Obtenir le bon état de controle pour la porte multi-controle X
    ctrl_state = get_disjunction_control_state(disj_clause)
    
    # Créer une porte multi-controle X avec le bon nombre de qubits
    mc_xgate = XGate().control(num_ctrl_qubits=nb_disj_variables, ctrl_state = ctrl_state)
    disj_qc.append(mc_xgate, qubits)
    
    # Ajouter une porte X au qubit à la position -1 (dernière), le qubit ancillaire
    disj_qc.x(qubits[-1])
    
    # Transformer le circuit de disjonction en une porte avec un nom, mcx
    disj_gate = disj_qc.to_gate(label='mcx')
    return disj_gate

Voyons comment on peut utiliser cette fonction.

In [ ]:
clause_test = {'x0' : True, 'x1' : True, 'x2' : False}

test_qc = QuantumCircuit(4)
disj_gate = logical_disjunction_to_gate(clause_test)
test_qc.append(disj_gate,range(4))
test_qc.decompose().draw('mpl')

# Construire l'oracle

In [ ]:
# Créer des registres quantiques pour les variables et les clauses
var_qubits = QuantumRegister(nb_variables, name='x')
clause_qubits = QuantumRegister(nb_clauses, name='c')

In [ ]:
# Construire le circuit de clauses
clauses_circuit = QuantumCircuit(var_qubits, clause_qubits)

# Ajouter chaque clause de disjonction comme une porte : 

# Clause 0
# Convertir la première disjonction en porte
gate = logical_disjunction_to_gate(clauses[0])
# Sélectionner les qubits associés à la première clause
c_qubits = get_disjunction_qubits(clauses[0], clause_qubits[0], var_qubits)
# Ajouter la porte au circuit de clauses
clauses_circuit.append(gate, c_qubits)

# Clause 1
# Écrire le code pour ajouter la clause 1 ici #

# Clause 2
# Écrire le code pour ajouter la clause 2 ici #


# ...

# Clause 6
# Écrire le code pour ajouter la clause 6 ici #


# P.S : vous pouvez utiliser des boucles si vous le désirez.  

# Afficher le circuit : 
clauses_circuit.decompose(gates_to_decompose=['mcx'], reps=2).draw(output='mpl')

In [ ]:
# Construire le circuit de l'oracle
oracle_circuit = QuantumCircuit(var_qubits, clause_qubits)

# Ajouter le circuit de clauses 
# --- la transformation vers une porte n'est utile qu'à des fins d'affichage plus tard ---
oracle_circuit.append(clauses_circuit.to_gate(label='clauses_circuit'), clauses_circuit.qubits)

# Ajouter la porte multi-controle Z
mc_z_gate = ZGate().control(nb_clauses - 1)
oracle_circuit.append(mc_z_gate, clause_qubits)

# Ajouter l'inverse du circuit de clauses
oracle_circuit.append(clauses_circuit.reverse_ops().to_gate(label='clauses_circuit'), oracle_circuit.qubits)

# Afficher le circuit
oracle_circuit.decompose(gates_to_decompose=['clauses_circuit', 'mcx'], reps=2).draw(output='mpl')


# Construire le diffuseur

Complétez le circuit du diffuseur.

In [ ]:
# Construire le circuit de diffuseur
diffuser_circuit = QuantumCircuit(var_qubits)

# Ajouter des portes H et X pour chaque qubit du diffuseur
### 4 portes H pour les variables
### 4 portes X pour les variables

# Ajouter une multi-controle Z  
mc_z_gate = ZGate().control(nb_variables - 1)
diffuser_circuit.append(mc_z_gate, var_qubits)

# Ajouter des portes X et H pour chaque qubit du diffuseur
### 4 portes X pour les variables
### 4 portes H pour les variables

# Afficher le circuit
diffuser_circuit.draw(output='mpl')

# Assembler le circuit de Grover

Complétez le circuit de Grover.

In [ ]:
# Construire le circuit de Grover
c_bits = ClassicalRegister(nb_variables)
grover_circuit = QuantumCircuit(var_qubits, clause_qubits, c_bits)

# Ajouter des portes H pour chaque variable
### 4 portes H pour les variables

# Identifier le nombre d'iterations
nb_iterations = 1 # Jouez avec le nombre d'itérations pour en voir l'effet

# Ajouter autant d'oracles et de diffuseurs qu'il y a de nombre d'itérations
for it in range(nb_iterations):
    grover_circuit.append(oracle_circuit.to_gate(label='oracle'), grover_circuit.qubits)
    grover_circuit.barrier(grover_circuit.qubits)
    grover_circuit.append(diffuser_circuit.to_gate(label='diffusor'), grover_circuit.qubits[0:nb_variables])
    
# Ajouter les mesures pour l'evaluation du circuit
grover_circuit.measure(var_qubits, c_bits)

# Afficher le circuit
grover_circuit.decompose(gates_to_decompose=['oracle', 'clauses_circuit', 'diffusor', 'mcx'], reps=3).draw(output='mpl',
                                                                                                   scale=0.8)

# Mesure de la solution

In [ ]:
from qiskit_aer import AerSimulator
from qiskit import transpile

# Prepaper une simulation pour rouler et mesurer la solution
def run_circuit(circ: QuantumCircuit) -> dict:
    """
    Run a quantum circuit on the AerSimulator and return the counts
    @param circ: QuantumCircuit to run
    @return: dictionary of measurement results and their counts
    """
    simulator = AerSimulator()
    circ = transpile(circ, simulator)
    result = simulator.run(circ, shots=1000).result()
    return result.get_counts(circ)

In [ ]:

# Executer le circuit et obtenir le compte de solutions
counts = run_circuit(grover_circuit)

print(counts)

In [ ]:
# Afficher l'histogramme de comptes
plot_histogram(counts)

Rappel : 

* x0 = Peureux
* x1 = Heureux
* x2 = Malade
* x3 = Bruyant